In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact, IntSlider, VBox
from IPython.display import display


with open('Data/nhl_season_2016.json', 'r') as f:
    data = json.load(f)['data']


rink_img_path = 'Data/rink.jpg'
rink_img = mpimg.imread(rink_img_path)



In [2]:
def plot_rink(game, event):
    """
    Plots the hockey rink and displays the location of a specific event on the rink.

    Args:
        game (dict): A dictionary containing game information, including game ID and season.
        event (dict): A dictionary containing event details, such as event coordinates 
                      ('xCoord', 'yCoord') and event description ('typeDescKey').
    
    The function uses the provided event data to plot the location of the event on the rink. 
    If the event has coordinates, it marks the position and labels it with a description.
    """
    fig, ax = plt.subplots(figsize=(8, 6))

    # Display the hockey rink image within the provided coordinates
    ax.imshow(rink_img, extent=[-100, 100, -42.5, 42.5])
    
    # Check if event contains x and y coordinates to plot
    if 'details' in event:
        if 'xCoord' in event['details'] and 'yCoord' in event['details']:
            x, y = event['details']['xCoord'], event['details']['yCoord']
            # Plot event position on the rink
            plt.scatter(x, y, color='blue', s=100)
            # Add event description text at the plotted position
            plt.text(x + 5, y, event['typeDescKey'], fontsize=12, color='white', 
                     bbox=dict(facecolor='black', alpha=0.5))
    
    # Set rink boundaries and aspect ratio
    ax.set_xlim(-100, 100)
    ax.set_ylim(-42.5, 42.5)
    ax.set_title(f"Game {game['id']} - {game['season']}")
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

def browse_events(game_idx, event_idx):
    """
    Displays information about a specific game and event, and visualizes the event on the rink.

    Args:
        game_idx (int): Index of the game in the dataset.
        event_idx (int): Index of the event within the game's play-by-play data.
    
    The function prints detailed information about the game (teams, score, shots on goal, overtime, 
    and shootout details) and then calls `plot_rink` to visualize the event on a rink plot.
    """
    # Fetch the specific game and event from the data
    game = data[game_idx]
    event = game['plays'][event_idx]
    
    # Print basic game information
    print("Game ID:", game['id'])
    print("Season:", game['season'])
    print(f"Date and Time: {game['gameDate']} at {game['startTimeUTC']}")

    home_team = game['homeTeam']
    away_team = game['awayTeam']
    
    # Print teams, score, and shots on goal (SoG) statistics
    print(f"Teams: {home_team['abbrev']} (Home) vs {away_team['abbrev']} (Away)")
    print(f"Goals: {home_team['score']} (Home) - {away_team['score']} (Away)")
    print(f"SoG: {home_team['sog']} (Home) - {away_team['sog']} (Away)")
    
    # Check if the game went to overtime or shootout
    if game['otInUse']:
        print("Overtime: Yes")
    else:
        print("Overtime: No")
        
    if game['shootoutInUse']:
        print("Shootout: Yes")
        print(f"SO Goals: {home_team.get('shootoutGoals', 'None')} (Home) - {away_team.get('shootoutGoals', 'None')} (Away)")
        print(f"SO Attempts: {home_team.get('shootoutAttempts', 'None')} (Home) - {away_team.get('shootoutAttempts', 'None')} (Away)")
    else:
        print("Shootout: No")

    # Plot the event on the rink and print the event details
    plot_rink(game, event)
    print(event)


In [3]:
max_games = len(data) -1  
game_slider = IntSlider(min=0, max=max_games, step=1, description='Game ID')
event_slider = IntSlider(min=0, max=200, step=1, description='Event')


# Use the sliders to interactively update the plot based on selected game and event
@interact(game_idx=game_slider, event_idx=event_slider)
def update_plot(game_idx, event_idx):
    event_slider.max = len(data[game_idx]['plays'])-1
    browse_events(game_idx, event_idx)


interactive(children=(IntSlider(value=0, description='Game ID', max=1229), IntSlider(value=0, description='Eve…

Etape 3:

In [4]:
import requests
import pandas as pd

players= {}
def request(url, id):
    """
    Sends a GET request to the specified URL and retrieves the response data.

    Args:
        url (str): The URL to request data from.
        id (str): The ID associated with the request (used for error reporting).
    
    Returns:
        dict or None: the function returns the JSON response as a dictionary. Otherwise, it returns None.
    """
    response = requests.get(url)

    if response.status_code == 200:
        return response.json() 
    else:
        print(f"Failed to fetch data for {id}. Status code: {response.status_code}")
        return None
    
def get_player_name(id):
    """
    Retrieves the first name of the player associated with the given player ID.

    Args:
        id (str): The unique player ID used to fetch player data from the NHL API.
    
    Returns:
        str or None: The first name of the player if found, otherwise None.
    
    The function checks if the player data is already cached in the `players` dictionary. 
    If not, it fetches the player data from the NHL API and stores it in the cache.
    """
    url = f"https://api-web.nhle.com/v1/player/{id}/landing"
    response = None
    if id not in players:
        response = request(url, id)
        if response is not None: 
            players[id] = response
            response = response['firstName']['default']
    else:
        response = players[id]['firstName']['default']
 
    return response

def get_force(situation_code, event_owner_team_id, home_team_id, away_team_id):
    """
    Determines the game situation (e.g., Power Play, Shorthanded, Even Strength) 
    based on the number of skaters and goalies for both teams.

    Args:
        situation_code (str): A 4-digit string representing the game situation:
                              - 1st digit: number of away goalies.
                              - 2nd digit: number of away skaters.
                              - 3rd digit: number of home skaters.
                              - 4th digit: number of home goalies.
        event_owner_team_id (str): The team ID associated with the event being analyzed.
        home_team_id (str): The team ID for the home team.
        away_team_id (str): The team ID for the away team.
    
    Returns:
        str: A string describing the game situation (e.g., "Power Play", "Shorthanded", 
             "Even Strength", or "Empty Net Power Play"). Returns "Unknown" if the situation
             cannot be determined.
    """
    away_goalie = int(situation_code[0])
    away_skaters = int(situation_code[1])
    home_skaters = int(situation_code[2])
    home_goalie = int(situation_code[3])

    # Check if both teams have an equal number of skaters
    if away_skaters == home_skaters:
        return "Even Strength"
    elif away_skaters > home_skaters:
        if event_owner_team_id == away_team_id:
            return "Power Play" if away_goalie == 1 else "Empty Net Power Play"
        else:
            return "Shorthanded"
    elif home_skaters > away_skaters:
        if event_owner_team_id == home_team_id:
            return "Power Play" if home_goalie == 1 else "Empty Net Power Play"
        else:
            return "Shorthanded"
    else:
        return "Unknown"


In [5]:
plays_data = []


for game in data:
    home_team = game['homeTeam']['name']['default']
    away_team = game['awayTeam']['name']['default']

    dfPlays = pd.DataFrame.from_records(game['plays'])
    df_filtered = dfPlays[dfPlays['typeDescKey'].isin(['goal', 'shot-on-goal'])]
    for idx, row in df_filtered.iterrows():
        
        event_id = row['eventId']
        time_in_period = row['timeInPeriod']
        period_number = row['periodDescriptor']['number']
        situation_code = row['situationCode']
        x_coord = row['details'].get('xCoord', None)
        y_coord = row['details'].get('yCoord', None)
        shot_type = row['details'].get('shotType', None)
        shooter_id = get_player_name(row['details'].get('shootingPlayerId', row['details'].get('scoringPlayerId', None)))
        goalie_id = get_player_name(row['details'].get('goalieInNetId', None))
        event_owner_team_id = row['details']['eventOwnerTeamId']
        event_type = row['typeDescKey']


        
        force_type = get_force(situation_code, event_owner_team_id, game['homeTeam']['id'], game['awayTeam']['id'])

        
        if event_owner_team_id == game['homeTeam']['id']:
            empty_net = goalie_id is None or int(situation_code[0]) == 0
        else:
            empty_net = goalie_id is None or int(situation_code[3]) == 0

        team = home_team if event_owner_team_id == game['homeTeam']['id'] else away_team
        
        plays_data.append({
            'Time': time_in_period,
            'Period': period_number,
            'Event ID': event_id,
            'Team': team,
            'Type': event_type,
            'Coordinates': (x_coord, y_coord),
            'Shooter ID': shooter_id,
            'Goalie ID': goalie_id,
            'Shot Type': shot_type,
            'Empty Net': empty_net,
            'Force Type': force_type
        })


df_final = pd.DataFrame(plays_data)





Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status code: 404
Failed to fetch data for None. Status co

In [6]:
df_final.head()

,Time,Period,Event ID,Team,Type,Coordinates,Shooter ID,Goalie ID,Shot Type,Empty Net,Force Type
0,01:10,1,57,Oilers,goal,"(72, 0)",Patrick,Brian,tip-in,False,Even Strength
1,01:45,1,61,Flames,shot-on-goal,"(-86, -7)",Alex,Cam,wrap-around,False,Even Strength
2,01:46,1,62,Flames,goal,"(-83, -5)",Alex,Cam,wrist,False,Even Strength
3,04:57,1,69,Flames,shot-on-goal,"(-58, -5)",TJ,Cam,snap,False,Shorthanded
4,05:34,1,70,Oilers,goal,"(57, -12)",Tyler,Brian,snap,False,Even Strength


In [8]:
df_final.to_csv('Data/partie3.csv', index=False)